In [1]:
import argparse
import pandas as pd
import numpy as np
from numpy import savez_compressed, load
import itertools
import re
import time
import os
import pickle

import datasets
from datasets import load_dataset, load_metric, Dataset, concatenate_datasets,DatasetDict
from datasets import load_from_disk
from tqdm import tqdm
tqdm.pandas(position=0,leave=True)

import transformers

from transformers import (
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    get_linear_schedule_with_warmup,
)
print("Transformers version is {}".format(transformers.__version__))

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from IPython.display import display, HTML

sns.set(style="whitegrid",palette='muted',font_scale=1.2)
rcParams['figure.figsize']=16,10

%config InlineBackend.figure_format="retina"
%matplotlib inline

pd.set_option('display.max_columns', None,'display.max_rows',None)

Transformers version is 4.6.1


In [2]:
input_dir="s3://trident-retention-output/multi-class"
train_df=pd.read_csv(os.path.join(input_dir,"train_df.csv"))
val_df=pd.read_csv(os.path.join(input_dir,"val_df.csv"))
test_df=pd.read_csv(os.path.join(input_dir,"test_df.csv"))

In [3]:
train_df.drop(['Unnamed: 0'],axis=1,inplace=True)
val_df.drop(['Unnamed: 0'],axis=1,inplace=True)
test_df.drop(['Unnamed: 0'],axis=1,inplace=True)
train_df.head()

,unum_id,year,month,Subtype,TextBody,new_category
0,571026399,2021,6,claim status,"hi, i submitted a claim fothe death of calvin ...",claim_issue
1,852810,2018,4,new plan administrator,we just need to make suthat payroll paycoknows...,new_plan_admin
2,462037348,2019,11,eoi submission,attached is the eoi foambehipp with fbt bank a...,eoi_issue
3,665488334,2020,11,employee coding,"good day, please process the terminations as f...",other-category
4,952809005,2021,2,invoice,"hi natasha, it was a pleasuspeaking with you t...",other-category


In [4]:
hf_train=Dataset.from_pandas(train_df)
hf_val=Dataset.from_pandas(val_df)
hf_test=Dataset.from_pandas(test_df)

hf_data=DatasetDict({"train":hf_train, "val":hf_val,  "test":hf_test})

In [5]:
hf_data

DatasetDict({
    train: Dataset({
        features: ['unum_id', 'year', 'month', 'Subtype', 'TextBody', 'new_category'],
        num_rows: 151610
    })
    val: Dataset({
        features: ['unum_id', 'year', 'month', 'Subtype', 'TextBody', 'new_category'],
        num_rows: 18951
    })
    test: Dataset({
        features: ['unum_id', 'year', 'month', 'Subtype', 'TextBody', 'new_category'],
        num_rows: 18949
    })
})

In [8]:
def label_distribution(df,col):
    tempt1=pd.DataFrame(df[col].value_counts(dropna=False)).reset_index().rename(columns={'index':col,col:'count'})
    tempt2=pd.DataFrame(df[col].value_counts(dropna=False,normalize=True)).reset_index().rename(columns={'index':col,col:'percentage'})
    return tempt1.merge(tempt2, on=col, how="inner")

def style_format(df, col, data_type="Training set"):
    return df.style.format({'count':'{:,}','percentage':'{:.2%}'})\
           .set_caption(f"{data_type} {col} distribution")\
           .set_table_styles([{'selector': 'caption','props': [('color', 'red'),('font-size', '15px')]}])
    

In [9]:
label_train=label_distribution(train_df,col="new_category")
style_format(label_train,col="new_category",  data_type="Training set")

,new_category,count,percentage
0,claim_issue,"54,270",35.80%
1,eoi_issue,"45,083",29.74%
2,other-category,"30,322",20.00%
3,new_plan_admin,"12,490",8.24%
4,billing_issue,"9,445",6.23%


In [10]:
label_train=label_distribution(test_df,col="new_category")
style_format(label_train,col="new_category",  data_type="Test set")

,new_category,count,percentage
0,claim_issue,"6,783",35.80%
1,eoi_issue,"5,635",29.74%
2,other-category,"3,790",20.00%
3,new_plan_admin,"1,561",8.24%
4,billing_issue,"1,180",6.23%


In [11]:
model_checkpoint="allenai/longformer-base-4096"
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
def train_test_data(df_train, df_test, feature_cols):
    train_df=Dataset.from_pandas(df_train)
    train_df=train_df.filter(lambda x: x[feature_cols]!=None)
    train_df=train_df.map(lambda x: tokenizer(x[feature_cols]),batched=True)

    test_df=Dataset.from_pandas(df_test)
    test_df=test_df.filter(lambda x: x[feature_cols]!=None)
    test_df=test_df.map(lambda x: tokenizer(x[feature_cols]),batched=True)
    
    def compute_lenth(example):
        return {"text_length":len(example["input_ids"])}
    
    train_df=train_df.map(compute_lenth)
    test_df=test_df.map(compute_lenth)
    
    return train_df, test_df

In [13]:
train_df1, test_df1=train_test_data(train_df, test_df, feature_cols="TextBody")

  0%|          | 0/152 [00:00<?, ?ba/s]

  0%|          | 0/152 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4482 > 4096). Running this sequence through the model will result in indexing errors


  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/151610 [00:00<?, ?ex/s]

  0%|          | 0/18949 [00:00<?, ?ex/s]

In [14]:
def statistics_compute(hf_df1,hf_df2,p=1):

    X=[]
    X.append(np.percentile(hf_df1['text_length'],p))
    X.append(np.percentile(hf_df2['text_length'],p))
    
    result={}
    result['percentile']=X
    result["min"]=[np.min(hf_df1['text_length']),np.min(hf_df2['text_length'])]
    result["max"]=[np.max(hf_df1['text_length']),np.max(hf_df2['text_length'])]
    result["mean"]=[np.mean(hf_df1['text_length']),np.mean(hf_df2['text_length'])]
    return result

def statistics_table(hf_df1,hf_df2):
    dict_data={}
    dict_data["data_type"]=["training", "test"]
    dict_data["# of obs"]=[len(hf_df1['text_length']),len(hf_df2['text_length'])]
    dict_data["Min of tokens"]=statistics_compute(hf_df1, hf_df2)["min"]
    dict_data["1% of tokens"]=statistics_compute(hf_df1, hf_df2, p=1)['percentile']
    dict_data["5% of tokens"]=statistics_compute(hf_df1, hf_df2, p=5)['percentile']
    dict_data["10% of tokens"]=statistics_compute(hf_df1, hf_df2, p=10)['percentile']
    dict_data["25% of tokens"]=statistics_compute(hf_df1, hf_df2, p=25)['percentile']
    dict_data["Median of tokens"]=statistics_compute(hf_df1, hf_df2, p=50)['percentile']
    dict_data["Average tokens"]=statistics_compute(hf_df1, hf_df2)["mean"]
    dict_data["75% of tokens"]=statistics_compute(hf_df1, hf_df2, p=75)['percentile']
    dict_data["90% of tokens"]=statistics_compute(hf_df1, hf_df2, p=90)['percentile']
    dict_data["95% of tokens"]=statistics_compute(hf_df1, hf_df2, p=95)['percentile']
    dict_data["99% of tokens"]=statistics_compute(hf_df1, hf_df2, p=99)['percentile']
    dict_data["Max of tokens"]=statistics_compute(hf_df1, hf_df2)["max"]
    token_count_df=pd.DataFrame(dict_data)
    return token_count_df

def style_format(token_count_df,  textbody="Full_TextBody"):
    token_count_df=token_count_df.set_index("data_type")
    token_count_df[list(token_count_df.columns)] = token_count_df[list(token_count_df.columns)].astype(int)
    return token_count_df.style.format("{:,}").set_caption(f"Summary Statistics of token lengths for {textbody} ").set_table_styles([{
        'selector': 'caption',
        'props': [
            ('color', 'red'),
            ('font-size', '20px')
        ]
    }])

In [15]:
token_count_df=statistics_table(train_df1,test_df1)
style_format(token_count_df,  textbody="Full_TextBody")

,# of obs,Min of tokens,1% of tokens,5% of tokens,10% of tokens,25% of tokens,Median of tokens,Average tokens,75% of tokens,90% of tokens,95% of tokens,99% of tokens,Max of tokens
data_type,,,,,,,,,,,,,
training,"151,610",8,62,92,122,199,312,424,496,809,"1,102","2,003","81,739"
test,"18,949",14,61,92,122,203,316,432,511,823,"1,108","2,017","31,241"
